# Analyse des marnages de réservoirs EPANET

## Importations

### Modèle EPANET

In [1]:
# Importations EPANET
import ganessa.en2emu as sim_et
from ganessa.en2emu import *
ET = sim_et.et

	Epanet version is: 2.2 (from owa-epanet 2.2.4)


### Interface graphique

In [2]:
# Importations de l'interface utilisateur
import os
from ipywidgets import Text
from IPython.display import display
#import argparse

## Paramétrage simulation EPANET

### Sélection modèle

In [3]:
# Widget de saisie
text_chooser = Text(
    value='modele.inp',
    placeholder='Entrer le chemin complet du fichier .inp',
    description='Fichier .inp :',
    disabled=False,
    continuous_update=False
)

# Affichage du widget
display(text_chooser)

Text(value='modele.inp', continuous_update=False, description='Fichier .inp :', placeholder='Entrer le chemin …

### Paramètres de la simulation

In [4]:
# Widget de saisie de la durée de simulation
text_duration = Text(
    value='24',
    placeholder='Entrer la durée de simulation',
    description='Durée (h) :',
    disabled=False,
    continuous_update=False
)

# Affichage du widget
display(text_duration)

Text(value='24', continuous_update=False, description='Durée (h) :', placeholder='Entrer la durée de simulatio…

### Vérifications inputs utilisateur

In [5]:
# Stockage des inputs dans des variables
model_file = text_chooser.value
duration = int(text_duration.value)

# Créer les fichiers de sortie .rpt et .out
rpt_file = model_file.replace('.inp', '.rpt')
out_file = model_file.replace('.inp', '.out')

# Vérification de l'existence et de la validité du fichier .inp
if os.path.isfile(model_file):
    print(f"Le fichier {model_file} existe.")
    # Vérification de l'extension .inp
    if model_file.endswith('.inp'):
        print(f"Le fichier {model_file} est un fichier .inp valide.")
    else:
        print(f"Le fichier {model_file} n'est pas un fichier .inp valide.")
else:
    print(f"Le fichier {model_file} n'existe pas.")

# Vérification durée de simulation
if duration <= 0:
    print(f"Durée de simulation invalide : {duration} heures")
else:
    print(f"Durée de simulation valide : {duration} heures")

Le fichier modele.inp existe.
Le fichier modele.inp est un fichier .inp valide.
Durée de simulation valide : 24 heures


## Initialisation du modèle EPANET

In [6]:
# Initialisation du modèle EPANET
try:
    model = initialize_model(model_file, rpt_file, out_file)
    if not model:
        raise Exception("Model initialization failed.")
    print("Model initialized successfully.")
except Exception as e:
    print(f"Error initializing model: {e}")

Error initializing model: name 'initialize_model' is not defined


### Récupération des ID de réservoirs

In [7]:
# Récupération de la liste des tanks
try:
    print("Retrieving tank IDs...")
    model.update_indexes()  # Update tank and pipe indices
    tank_ids = [et.ENgetnodeid(ix) for ix in model.tank_index]  # Get tank IDs
    print("Tank IDs retrieved successfully.")
    
    # Display tank IDs
    for tank_id in tank_ids:
        print(f"Tank ID: {tank_id}")

except Exception as e:
    print(f"Error during tank ID retrieval: {e}")

Retrieving tank IDs...
Error during tank ID retrieval: name 'model' is not defined


## Lancement de la simulation

In [8]:
# Extraction des pressions
print("Extracting tank pressures...")
try:
    _, _, pressures, _ = model.runH_results_part([], tank_ids, tank_ids)
    tanks = {tank_id: pressures[tank_id] for tank_id in tank_ids}
    print("Tank pressure extraction complete.")
except Exception as e:
    print(f"Error during pressure extraction: {e}")

Extracting tank pressures...
Error during pressure extraction: name 'model' is not defined


## Calculs des marnages des réservoirs sur 24h

In [9]:
# Calcul des marnages
stats = {}
for tank_id, pressures in tanks.items():
    if pressures:
        stats[tank_id] = {
            "max_pressure": max(pressures),
            "min_pressure": min(pressures),
        }
    else:
        stats[tank_id] = {
            "max_pressure": None,
            "min_pressure": None,
        }

NameError: name 'tanks' is not defined

# Affichage des stats finales

In [ ]:
print("\n=== Tank Pressure Statistics ===")

# Affichage des statistiques
analysis_file_path = os.path.join(os.path.dirname(model_file), "analysis.txt")
with open(analysis_file_path, "w") as analysis_file:
    for tank_id, stat in stats.items():
        line = f"Tank {tank_id}: Max = {stat['max_pressure']}, Min = {stat['min_pressure']}\n"
        print(line.strip())
        analysis_file.write(line)

print(f"\nAnalysis saved to {analysis_file_path}")

# Fermeture du modèle
if 'model' in locals() and model:
    model.close()


=== Tank Pressure Statistics ===
Tank ABBARETZ: Max = 5.962420664756762, Min = 3.870002974194155
Tank AVESSAC: Max = 8.699999999999998, Min = 2.999999999999994
Tank BACHEBIROCHERE: Max = 4.947451085767621, Min = 2.876579436929387
Tank BACHECHALOUSSIERE: Max = 5.100000000000001, Min = 2.9940584908617542
Tank BACHEGESVRINE: Max = 4.999999999999998, Min = 2.8706686017187057
Tank BACHEPOINTDUJ: Max = 4.904376165938177, Min = 2.9999999999999982
Tank BACHEROGERE: Max = 13.000000000000004, Min = 10.815923255258353
Tank BARRELECELLIER: Max = 3.9999999999999916, Min = 1.6544001663195946
Tank BASSEGOULAINEDEUX: Max = 4.902844081421119, Min = 0.07674872653897782
Tank BASSEGOULAINEUN: Max = 3.841397026008484, Min = 1.9999999999999996
Tank BAZOUINIEREUNETDEUX: Max = 5.900000000000001, Min = 4.8151333388181214
Tank BEHORAIS: Max = 11.93682200587594, Min = 8.5898064698809
Tank BELAIR: Max = 10.322244882341357, Min = 2.835969180940601
Tank BERE: Max = 5.000000000000007, Min = 2.999999999999994
Tank B